In [8]:
!pip install requests beautifulsoup4
!pip install langchain
!pip install unstructured
!pip install faiss-cpu
!pip install sentence-transformers
pip install -U langchain-community faiss-cpu langchain-openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 15.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.3/410.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.0 MB/s eta 0:00:0000:0100:01


In [5]:
pip install langchain_community.document_loaders


ERROR: Could not find a version that satisfies the requirement langchain_community.document_loaders (from versions: none)
ERROR: No matching distribution found for langchain_community.document_loaders
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import requests
from bs4 import BeautifulSoup
from langchain_community.document_loaders import DirectoryLoader

def extract_info_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        info_text = '\n'.join(p.text for p in paragraphs)
        return info_text
    else:
        print("Failed to retrieve data from URL:", url)
        return None

def save_to_text_file(text, filename, folder='extracted_info'):
    if not os.path.exists(folder):
        os.makedirs(folder)
    file_path = os.path.join(folder, filename)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)

if __name__ == "__main__":
    list_of_URLS = [
        'https://catalogue.uci.edu/allcourses/compsci/',
        'https://catalogue.uci.edu/donaldbrenschoolofinformationandcomputersciences/departmentofcomputerscience/computerscience_bs/#sampleprogramtext',
        'https://catalogue.uci.edu/donaldbrenschoolofinformationandcomputersciences/departmentofcomputerscience/computerscience_bs/#requirementstext'
    ]
    for url in list_of_URLS: 
        info_text = extract_info_from_url(url)
        print("Extracted info from {}".format(url))
        if info_text:
            file_name = 'extracted_info_{}.txt'.format(url.split('/')[-2])
            save_to_text_file(info_text, file_name)
            print("Information extracted and saved to '{}'".format(file_name))
    loader = DirectoryLoader('extracted_info', '*.txt')


Extracted info from https://catalogue.uci.edu/allcourses/compsci/
Information extracted and saved to 'extracted_info_compsci.txt'
Extracted info from https://catalogue.uci.edu/donaldbrenschoolofinformationandcomputersciences/departmentofcomputerscience/computerscience_bs/#sampleprogramtext
Information extracted and saved to 'extracted_info_computerscience_bs.txt'
Extracted info from https://catalogue.uci.edu/donaldbrenschoolofinformationandcomputersciences/departmentofcomputerscience/computerscience_bs/#requirementstext
Information extracted and saved to 'extracted_info_computerscience_bs.txt'


TypeError: 'DirectoryLoader' object is not subscriptable

In [6]:
for i in range(1): 
    print(loader)

In [10]:
import getpass
hugging_face_api_key = getpass.getpass(prompt="Enter your Hugging Face API key: ")

Enter your Hugging Face API key:  ········


In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
import numpy as np
from langchain.storage import InMemoryStore
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

def returnSplits():
    loader=DirectoryLoader('extracted_info', show_progress=True, use_multithreading=True)
    documents=loader.load()
    return documents


def createVectorStore():
    embeddings = HuggingFaceEmbeddings(model_name='WhereIsAI/UAE-Large-V1')
    store = InMemoryStore()
    vectorstore = FAISS.from_documents(returnSplits(), embedding=embeddings)
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=512)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)
    retriever = ParentDocumentRetriever(
        vectorstore=vectorstore,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter
    )
    retriever.add_documents(returnSplits())
    final_path='faissDB'
    vectorstore.save_local(final_path)
    print("==== CREATED VECTOR DATABASE ===")
    return vectorstore

vectorstore = createVectorStore()

You try to use a model that was created with version 2.5.1, however, your version is 2.2.2. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





TypeError: Pooling.__init__() got an unexpected keyword argument 'pooling_mode_weightedmean_tokens'

In [13]:
def loadVectorStore():
    final_path='faissDB'
    embeddings=HuggingFaceEmbeddings(model_name='WhereIsAI/UAE-Large-V1')
    vectorstore = FAISS.load_local(final_path,embeddings,allow_dangerous_deserialization=True)
    return vectorstore

vectorstore = loadVectorStore()